# TP Python

PySpark est une interface pour Apache Spark en Python, permettant de traiter de grandes quantités de données en parallèle sur des clusters, en combinant la puissance de calcul de Spark avec la simplicité de Python.

## Déploiement

Le déploiement se fait comme pour les TP précédents, à l'aide de Docker Compose. N'oubliez pas de lancer Docker Desktop en premier lieu !
```bash 
docker compose up --build
```

Sont déployés :
- un master Spark 
- un worker Spark
- une installation de [Minio](https://min.io/) servant au stockage des données accédée par Spark
- une installation de Jupyter comprenant les bibliothèques nécessaires pour le TP

L'UI de Spark est disponible à l'adresse suivante : http://localhost:8080.
Nous utilisons la solution de stockage objet Minio, accessible à l'adresse suivante : http://localhost:19001.

### Problèmes possibles

Quelques messages d'erreurs que vous pouvez rencontrer, et comment les gérer :
- Message d'erreur "Cannot run multiple SparkContexts at once" : vous ne pouvez initialiser la connexion avec Spark qu'une fois par notebook, la solution est simplement de faire reset du notebook (bouton restart en haut du notebook).
- Plus d'exécuteur disponible dans Spark : un job est problablement déjà en cours. Coupez le sur l'[interface de Spark](http://127.0.0.1:8080) (ou coupez l'ensemble de l'installation avec `docker compose down`), puis faites un reset du notebook (bouton restart en haut du notebook).
- L'option restart est grisée : fermez l'onglet du notebook et ouvrez-le à nouveau

In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf() \
    .setAppName('SparkApp') \
    .setMaster('spark://spark:7077') \
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") # utilisé pour le stockage 
sc = SparkContext(conf=conf)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-07c9bc79-fcec-47b0-8466-44def05a1067;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.4!hadoop-aws.jar (114ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.12.262!aws-java-sdk-bundle.jar (102532ms)
downloading https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.0.7.Final/wildfly-openssl-1.0.7.Final.jar ...
	[SUCCESSFUL 

## Spark - RDD style

Voici quelques exemples.

In [2]:
# Create an RDD containing numbers from 1 to 1000
numbers_rdd = sc.parallelize(range(1, 1000))

# Count the elements in the RDD
count = numbers_rdd.count()
print(f"Count of numbers from 1 to 1000 is: {count}")

Count of numbers from 1 to 1000 is: 999


### Calcul de moyenne

Quelques liens utiles pour comprendre :
- la fonction de création d'un RDD `parallelize` : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkContext.parallelize.html 
- la transformation `map` : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.map.html
- la transformation `reduceByKey` : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.reduceByKey.html
- la fonction `mapValues` : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.mapValues.html

In [3]:
# Create initial data in Python
data = [("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)]

# Create initial RDD from standard data
dataRDD = sc.parallelize(data)

# Filter out None values and log data
mapped_ages = dataRDD.map(lambda p: (str(p[0]), (int(p[1]), 1)))

# Reduce by key to sum ages and count per name
summed_ages = mapped_ages.reduceByKey(lambda p1, p2: (p1[0] + p2[0], p1[1] + p2[1]))

# Compute the average age per name
average_ages = summed_ages.mapValues(lambda v: v[0] / v[1])

# Collect the results
ages = average_ages.collect()
print("Array(", ", ".join([str(age) for age in ages]), ")")

Array( ('Brooke', 22.5), ('Denny', 31.0), ('Jules', 30.0), ('TD', 35.0) )


## Pagerank

PageRank est un algorithme développé par Google pour mesurer l'importance relative de chaque page web en fonction de son nombre et de la qualité des liens entrants. Il attribue un score de popularité à chaque page, influençant son classement dans les résultats de recherche.

Quelques liens en plus pour comprendre :
- la transformation `join` sur les RDD : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.join.html
- la transformation `flatmap` sur les RDD : https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.flatMap.html

N'hésitez pas à essayer de décomposer les calculs pour comprendre ! Attention à bien faire une action de type `collect` pour voir les résultats intermédiaires.

In [13]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

# Parameters
ITERATIONS, a, N = 10, 0.15, 5

# Inline example data: (Website, List[Linked Websites])
data = [
    ("google.com", ["facebook.com", "linkedin.com", "youtube.com"]),
    ("twitter.com", ["google.com", "youtube.com", "linkedin.com"]),
    ("facebook.com", ["google.com", "twitter.com"]),
    ("youtube.com", ["google.com", "twitter.com"]),
    ("linkedin.com", ["google.com", "twitter.com"])
]

links = sc.parallelize(data).partitionBy(8).persist()
print(links.collect())

# Initialize ranks: RDD of (Website, initial rank)
ranks = links.mapValues(lambda _: 1.0 / N)

print(links.join(ranks).collect())
print(links.join(ranks).collect()[1])
print(links.join(ranks).collect()[1][1])
print(links.join(ranks).collect()[1][0])

# PageRank Iterations
for _ in range(ITERATIONS):
    contribs = links.join(ranks).flatMap(
        lambda site_links_rank: [(dest, site_links_rank[1][1] / len(site_links_rank[1][0])) 
                                 for dest in site_links_rank[1][0]]
    )
    print(contribs.collect())
    ranks = contribs.reduceByKey(lambda x, y: x + y).mapValues(lambda total: a / N + (1 - a) * total)
    print(ranks.collect())
# Output final ranks
print(ranks.collect())

[('google.com', ['facebook.com', 'linkedin.com', 'youtube.com']), ('linkedin.com', ['google.com', 'twitter.com']), ('facebook.com', ['google.com', 'twitter.com']), ('twitter.com', ['google.com', 'youtube.com', 'linkedin.com']), ('youtube.com', ['google.com', 'twitter.com'])]


[('google.com', (['facebook.com', 'linkedin.com', 'youtube.com'], 0.2)), ('linkedin.com', (['google.com', 'twitter.com'], 0.2)), ('facebook.com', (['google.com', 'twitter.com'], 0.2)), ('twitter.com', (['google.com', 'youtube.com', 'linkedin.com'], 0.2)), ('youtube.com', (['google.com', 'twitter.com'], 0.2))]


('linkedin.com', (['google.com', 'twitter.com'], 0.2))


(['google.com', 'twitter.com'], 0.2)


linkedin.com


...

[('facebook.com', 0.09800428259516862), ('linkedin.com', 0.09800428259516862), ('youtube.com', 0.09800428259516862), ('google.com', 0.0910393176368373), ('twitter.com', 0.0910393176368373), ('google.com', 0.0574739172888284), ('twitter.com', 0.0574739172888284), ('google.com', 0.07562734902982934), ('youtube.com', 0.07562734902982934), ('linkedin.com', 0.07562734902982934), ('google.com', 0.0910393176368373), ('twitter.com', 0.0910393176368373)]


[('google.com', 0.2979029163534824), ('linkedin.com', 0.17758688688124824), ('facebook.com', 0.11330364020589333), ('twitter.com', 0.23361966967812756), ('youtube.com', 0.17758688688124824)]


[('facebook.com', 0.09930097211782747), ('linkedin.com', 0.09930097211782747), ('youtube.com', 0.09930097211782747), ('google.com', 0.08879344344062412), ('twitter.com', 0.08879344344062412), ('google.com', 0.056651820102946664), ('twitter.com', 0.056651820102946664), ('google.com', 0.07787322322604252), ('youtube.com', 0.07787322322604252), ('linkedin.com', 0.07787322322604252), ('google.com', 0.08879344344062412), ('twitter.com', 0.08879344344062412)]


[('google.com', 0.2952951406787018), ('linkedin.com', 0.18059806604228948), ('facebook.com', 0.11440582630015335), ('twitter.com', 0.22910290093656566), ('youtube.com', 0.18059806604228948)]


[('google.com', 0.2952951406787018), ('linkedin.com', 0.18059806604228948), ('facebook.com', 0.11440582630015335), ('twitter.com', 0.22910290093656566), ('youtube.com', 0.18059806604228948)]


## Spark + Minio

Nous allons utiliser le stockage objet Minio pour héberger les données de notre installation Spark.

MinIO est une solution de stockage objet haute performance, compatible avec l'API S3 d'AWS, permettant de gérer des données non structurées à grande échelle. Il est conçu pour des environnements cloud, hybrides ou sur site, offrant une infrastructure de stockage distribuée et évolutive.

La copie de fichier peut se faire par la bibliothèque Minio Python, ou alors par le biais de l'UI Web, accessible sur http://localhost:19001. Les identifiants sont "root" et "password" (on peut les retrouver dans le fichier [docker-compose.yml](docker-compose.yml)). 

Le principe général de MinIO (comme pour les autres systèmes de stockage objet tels qu'AWS S3) est d'organiser les données en buckets, qui sont des conteneurs virtuels pour le stockage de fichiers ou d’objets. Chaque bucket est unique dans le système et peut contenir un nombre illimité d'objets, identifiés par des clés uniques. 

### Exercice: Moby Dick

Prérequis : le contenu du livre est présent dans le fichier `pg2701.txt` ([lien internet](https://nyu-cds.github.io/python-bigdata/files/pg2701.txt)) du TP4.

In [14]:
!wget https://nyu-cds.github.io/python-bigdata/files/pg2701.txt

--2025-02-25 10:47:03--  https://nyu-cds.github.io/python-bigdata/files/pg2701.txt
Resolving nyu-cds.github.io (nyu-cds.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to nyu-cds.github.io (nyu-cds.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1257296 (1.2M) [text/plain]
Saving to: ‘pg2701.txt’

pg2701.txt          100%[===================>]   1.20M  6.95MB/s    in 0.2s    

2025-02-25 10:47:03 (6.95 MB/s) - ‘pg2701.txt’ saved [1257296/1257296]



In [15]:
# paramètres utilisés pour stockage
import docker

minio_ip_address = "minio"

# Décommenter dans le cas de WSL 
#network_name = "tp4_default"
#container_name = "minio"
#
#client = docker.from_env()
#container = client.containers.get(container_name)
#
#minio_ip_address: str = container.attrs['NetworkSettings']['Networks'][network_name]['IPAddress']
#minio_ip_address

sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", f"http://{minio_ip_address}:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "root")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "password")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")

In [16]:
# https://min.io/docs/minio/linux/developers/python/API.html

from minio import Minio
client_minio = Minio(
    f"{minio_ip_address}:9000",
    access_key="root",
    secret_key="password",
    secure=False
)

# Création du bucket tp5
if client_minio.bucket_exists("tp4") == False:
    client_minio.make_bucket("tp4")
client_minio.fput_object("tp4", "pg2701.txt", "pg2701.txt") # copie du fichier local dans le bucket

Minio reprend le principe du stockage cloud S3 : il permet de stocker des fichiers dans des "buckets". Les buckets dans MinIO sont des conteneurs de stockage pour organiser et gérer des objets (fichiers) de manière structurée, similaires aux dossiers dans un système de fichiers, mais optimisés pour le stockage objet.
Vérifiez que le fichier est bien présent dans le bucket `tp4` de Minio : [http://localhost:19001/browser/tp4/](http://localhost:19001/browser/tp4/). Vous pouvez utiliser l'utilisateur `root` et le mot de passe `password` pour vous connecter.

In [17]:
minio_file = "s3a://tp4/pg2701.txt"
# adresse du fichier dans le bucket minio
text = sc.textFile(minio_file) 
print(text.take(10))

25/02/25 10:47:18 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


['The Project Gutenberg EBook of Moby Dick; or The Whale, by Herman Melville', '', 'This eBook is for the use of anyone anywhere at no cost and with', 'almost no restrictions whatsoever.  You may copy it, give it away or', 're-use it under the terms of the Project Gutenberg License included', 'with this eBook or online at www.gutenberg.org', '', '', 'Title: Moby Dick; or The Whale', '']


### Exercice 1

1. Compter le nombre de mots total du livre.
2. Compter le nombre d'occurrences par mot, trier par nombre décroissant (prendre les 10 premiers).
3. Compter le nombre de mots par phrase.

In [18]:
text = sc.textFile(minio_file) 
count_words = text.flatMap(lambda x:x.split()) \
    .count()
print(f"Nombre de mots: {count_words}")

Nombre de mots: 215135


In [ ]:
text = sc.textFile(minio_file) 
count_by_words = text.flatMap(lambda x: x.split()) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x1, x2:  x1 + x2) \
    .sortBy(lambda x: x[1], ascending=False)
output = count_by_words.take(10)
print(f"Mots les plus employés: {output}")

Mots les plus employés: [('the', 13765), ('of', 6587), ('and', 5951), ('a', 4533), ('to', 4510), ('in', 3879), ('that', 2693), ('his', 2415), ('I', 1724), ('with', 1692)]


25/02/25 11:22:42 ERROR TaskSchedulerImpl: Lost executor 0 on 172.26.0.2: worker lost: Not receiving heartbeat for 60 seconds
25/02/25 11:22:42 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_667_1 !
25/02/25 11:22:42 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_667_6 !
25/02/25 11:22:42 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_408_5 !
25/02/25 11:22:42 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_778_2 !
25/02/25 11:22:42 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_667_0 !
25/02/25 11:22:42 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_778_6 !
25/02/25 11:22:42 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_577_4 !
25/02/25 11:22:42 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_778_3 !
25/02/25 11:22:42 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_667_5 !
25/02/25 11:22:42 WARN Block

In [ ]:
text = sc.textFile(minio_file)
phrases = text.flatMap(lambda ligne: ligne.split("."))
mots_par_phrase = phrases.map(lambda phrase: len(phrase.split()))
total_mots = mots_par_phrase.reduce(lambda a, b: a + b)
nombre_phrases = mots_par_phrase.count()
moyenne_mots_par_phrase = total_mots / nombre_phrases

print(f"Moyenne de mots par phrase : {moyenne_mots_par_phrase}")

Télécharger un autre livre (en trouver un sur https://www.gutenberg.org/browse/scores/top par exemple, télécharger au format "Plain Text UTF-8"), et lancer les jobs dessus.

In [ ]:
!wget https://www.gutenberg.org/cache/epub/345/pg345.txt

In [ ]:
client_minio.fput_object("tp4", "pg345.txt", "pg345.txt") # copie du fichier local dans le bucket

In [ ]:
minio_file = "s3a://tp4/pg345.txt"
text = sc.textFile(minio_file) 
count_words = text.flatMap(lambda x:x.split()) \
    .count()
print(f"Nombre de mots: {count_words}")
text = sc.textFile(minio_file) 
count_by_words = text.flatMap(lambda x: x.split()) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x1, x2:  x1 + x2) \
    .sortBy(lambda x: x[1], ascending=False)
output = count_by_words.take(10)
print(f"Mots les plus employés: {output}")
text = sc.textFile(minio_file)
phrases = text.flatMap(lambda ligne: ligne.split("."))
mots_par_phrase = phrases.map(lambda phrase: len(phrase.split()))
total_mots = mots_par_phrase.reduce(lambda a, b: a + b)
nombre_phrases = mots_par_phrase.count()
moyenne_mots_par_phrase = total_mots / nombre_phrases

print(f"Moyenne de mots par phrase : {moyenne_mots_par_phrase}")

### Exercice 2 : JSON

Charger le fichier `countries.json` ((adresse)[http://api.worldbank.org/v2/countries?per_page=304&format=json]), et chargez le dans Minio comme fait précédemment. 

Calculez ensuite le nombre de pays par niveau de revenu à l'aide d'un job Pyspark.

Vous pouvez vous aider des lignes suivantes en premier lieu (afin de se concentrer directement sur le tableau des pays, contenu dans le deuxième élément du tableau racine) :
```python
rdd = sc.textFile(f"s3a://tp4/countries.json") # chargement du fichier countries.json
mapped_rdd = rdd.map(lambda f: json.loads(f)) # chargement du fichier json dans un dictionnaire
country_rdd = mapped_rdd.flatMap(lambda x: x[1]) # on récupère le tableau des pays
```




In [ ]:
!wget -O countries.json  "http://api.worldbank.org/v2/countries?per_page=304&format=json"

In [ ]:
minio_file = "countries.json"
client_minio.fput_object("tp4", minio_file, minio_file)

In [ ]:
import json 
rdd = sc.textFile(f"s3a://tp4/{minio_file}")
mapped_rdd = rdd.map(lambda line: json.loads(line))
country_rdd = mapped_rdd.flatMap(lambda x: x[1])
print(country_rdd.take(10))
income_counts = country_rdd.map(lambda country: (country['incomeLevel']['value'], 1)) \
                           .reduceByKey(lambda a, b: a + b)
# Collect and print results
print("Number of countries by income level:")
for income_level, count in income_counts.collect():
    print(f"{income_level}: {count}")

## Exercice 3

Réalisez trois requêtes non triviales sur les fichiers json des [pokemon](https://github.com/fanzeyi/pokemon.json).
